# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,22.64,91,100,4.12,US,1697300478
1,1,bredasdorp,-34.5322,20.0403,16.41,78,83,9.67,ZA,1697300479
2,2,hamilton,39.1834,-84.5333,17.10,81,100,6.17,US,1697300214
3,3,badger,64.8000,-147.5333,-2.02,88,100,1.54,US,1697300016
4,4,bilibino,68.0546,166.4372,-10.58,97,100,0.98,RU,1697299995
...,...,...,...,...,...,...,...,...,...,...
556,556,poronaysk,49.2167,143.1167,7.38,93,14,3.22,RU,1697300688
557,557,ola,59.5833,151.2833,-1.44,69,21,2.62,RU,1697300688
558,558,port blair,11.6667,92.7500,26.02,94,40,0.00,IN,1697300688
559,559,usinsk,65.9939,57.5281,-1.07,98,74,2.24,RU,1697300689


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
City_map = city_data_df.hvplot.points('Lng','Lat',geo=True,color='City',tiles='OSM',frame_width=800,
    frame_height=600, size='Humidity', alpha=0.5)

# Display the map
City_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
                               & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_ideal_df.dropna()

# Display sample data
city_data_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,kashmar,35.2383,58.4656,21.98,35,0,3.09,IR,1697300497
111,111,perugorria,-29.3413,-58.6106,24.52,35,0,4.31,AR,1697300519
154,154,chandler,33.3062,-111.8412,26.19,28,0,4.12,US,1697300537
251,251,siwa oasis,29.2041,25.5195,26.47,31,0,3.54,EG,1697300571
425,425,simdega,22.6167,84.5167,22.72,63,0,1.70,IN,1697300630
427,427,sirte,31.2089,16.5887,24.52,58,0,3.01,LY,1697300631
430,430,valderice,38.0401,12.6151,24.05,73,0,3.09,IT,1697300632
441,441,callaway,30.1530,-85.5699,26.06,75,0,2.68,US,1697300635


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,kashmar,IR,35.2383,58.4656,35,
111,perugorria,AR,-29.3413,-58.6106,35,
154,chandler,US,33.3062,-111.8412,28,
251,siwa oasis,EG,29.2041,25.5195,31,
425,simdega,IN,22.6167,84.5167,63,
427,sirte,LY,31.2089,16.5887,58,
430,valderice,IT,38.0401,12.6151,73,
441,callaway,US,30.1530,-85.5699,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel


radius = 10000
params = {
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["apiKey"] = geoapify_key
    params["limit"] = 1
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
  
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kashmar - nearest hotel: خانه معلم کاشمر
perugorria - nearest hotel: No hotel found
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
siwa oasis - nearest hotel: فندق الكيلانى
simdega - nearest hotel: Amanlok
sirte - nearest hotel: فندق المدينة-سرت
valderice - nearest hotel: Hotel History
callaway - nearest hotel: Holiday Inn Express & Suites Panama City-Tyndall


,City,Country,Lat,Lng,Humidity,Hotel Name
48,kashmar,IR,35.2383,58.4656,35,خانه معلم کاشمر
111,perugorria,AR,-29.3413,-58.6106,35,No hotel found
154,chandler,US,33.3062,-111.8412,28,Crowne Plaza - San Marcos Golf Resort
251,siwa oasis,EG,29.2041,25.5195,31,فندق الكيلانى
425,simdega,IN,22.6167,84.5167,63,Amanlok
427,sirte,LY,31.2089,16.5887,58,فندق المدينة-سرت
430,valderice,IT,38.0401,12.6151,73,Hotel History
441,callaway,US,30.1530,-85.5699,75,Holiday Inn Express & Suites Panama City-Tyndall


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng','Lat',geo=True,color='City',tiles='OSM',frame_width=800,
    frame_height=600, hover_cols= ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)